## Question 3

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import csv
import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [2]:
pd.set_option('max_colwidth', 800)

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text 
soup = BeautifulSoup(source, 'lxml')

In [4]:
csv_file = open('toronto_postal_codes.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Postal Code', 'Borough', 'Neighbourhood'])

35

In [5]:
table = soup.find('table', class_ = 'wikitable sortable') # Gets the table from the webpage
rows = table.find_all('tr') # Gets the table rows

postcodes = [] # Initializes the raw postcodes list
boroughs = [] # Initializes the raw boroughs list
neighbourhoods = [] # Initializes the raw neighbourhoods list

for row in rows:    
    columns = row.find_all('td')
    try :
        if columns[1].text != 'Not assigned':  # To skip if the borough name is 'Not Assigned'
            
            postcode = columns[0].text.rstrip("\r\n")
            postcodes.append(postcode)
            
            borough = columns[1].text.rstrip("\r\n")
            boroughs.append(borough)
            
            neighbourhood = columns[2].text.split('\n')[0] # Removing the newline character at the end     
            
            if neighbourhood == 'Not assigned': # Assigning the same name to neighbourhood if it is 'Not Assigned'
                neighbourhood = borough            
                
            neighbourhoods.append(neighbourhood)
             
    except Exception as e : # To skip the first row which contains column names
        pass 
    
postcode_explored = [] # Initializing the list of explored postcodes
for index_i, postcode_i in enumerate(postcodes) :   
    if postcode_i not in postcode_explored :
        nbds = neighbourhoods[index_i]
        for index_f, postcode_f in enumerate(postcodes) :
            if postcode_i == postcode_f and index_i != index_f:
                nbds = nbds + ', ' + neighbourhoods[index_f] # Concatenating the neighbourhood names
        csv_writer.writerow([postcode_i, boroughs[index_i], nbds]) # Writing the rows in the csv file
        postcode_explored.append(postcode_i)

In [6]:
csv_file.close()

In [7]:
df = pd.read_csv('toronto_postal_codes.csv')

In [8]:
toronto_df = df[df.Borough != "Not assigned"].reset_index(drop=True)

In [9]:
postal_codes = df['Postal Code'].values

In [10]:
df_1 = pd.read_csv('Geospatial_Coordinates.csv')

In [11]:
Toronto_df = pd.merge(toronto_df,
                 df_1[['Postal Code','Latitude', 'Longitude']],
                 on='Postal Code')
Toronto_df

Postal Code           Borough  \
0           M3A        North York   
1           M4A        North York   
2           M5A  Downtown Toronto   
3           M6A        North York   
4           M7A  Downtown Toronto   
5           M9A         Etobicoke   
6           M1B       Scarborough   
7           M3B        North York   
8           M4B         East York   
9           M5B  Downtown Toronto   
10          M6B        North York   
11          M9B         Etobicoke   
12          M1C       Scarborough   
13          M3C        North York   
14          M4C         East York   
15          M5C  Downtown Toronto   
16          M6C              York   
17          M9C         Etobicoke   
18          M1E       Scarborough   
19          M4E      East Toronto   
20          M5E  Downtown Toronto   
21          M6E              York   
22          M1G       Scarborough   
23          M4G         East York   
24          M5G  Downtown Toronto   
25          M6G  Downtown Toronto   
26          M1H       Scarborough   
27          M2H        North York   
28          M3H        North York   
29          M4H         East York   
30          M5H  Downtown Toronto   
31          M6H      West Toronto   
32          M1J       Scarborough   
33          M2J        North York   
34          M3J        North York   
35          M4J         East York   
36          M5J  Downtown Toronto   
37          M6J      West Toronto   
38          M1K       Scarborough   
39          M2K        North York   
40          M3K        North York   
41          M4K      East Toronto   
42          M5K  Downtown Toronto   
43          M6K      West Toronto   
44          M1L       Scarborough   
45          M2L        North York   
46          M3L        North York   
47          M4L      East Toronto   
48          M5L  Downtown Toronto   
49          M6L        North York   
50          M9L        North York   
51          M1M       Scarborough   
52          M2M        North York   
53          M3M        North York   
54          M4M      East Toronto   
55          M5M        North York   
56          M6M              York   
57          M9M        North York   
58          M1N       Scarborough   
59          M2N        North York   
60          M3N        North York   
61          M4N   Central Toronto   
62          M5N   Central Toronto   
63          M6N              York   
64          M9N              York   
65          M1P       Scarborough   
66          M2P        North York   
67          M4P   Central Toronto   
68          M5P   Central Toronto   
69          M6P      West Toronto   
70          M9P         Etobicoke   
71          M1R       Scarborough   
72          M2R        North York   
73          M4R   Central Toronto   
74          M5R   Central Toronto   
75          M6R      West Toronto   
76          M7R       Mississauga   
77          M9R         Etobicoke   
78          M1S       Scarborough   
79          M4S   Central Toronto   
80          M5S  Downtown Toronto   
81          M6S      West Toronto   
82          M1T       Scarborough   
83          M4T   Central Toronto   
84          M5T  Downtown Toronto   
85          M1V       Scarborough   
86          M4V   Central Toronto   
87          M5V  Downtown Toronto   
88          M8V         Etobicoke   
89          M9V         Etobicoke   
90          M1W       Scarborough   
91          M4W  Downtown Toronto   
92          M5W  Downtown Toronto   
93          M8W         Etobicoke   
94          M9W         Etobicoke   
95          M1X       Scarborough   
96          M4X  Downtown Toronto   
97          M5X  Downtown Toronto   
98          M8X         Etobicoke   
99          M4Y  Downtown Toronto   
100         M7Y      East Toronto   
101         M8Y         Etobicoke   
102         M8Z         Etobicoke   

                                                                                                                              Neighbourhood  \
0           

In [12]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [13]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Borough'], Toronto_df['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [14]:
CLIENT_ID = 'OBQ0UO55QCKZIYTMPB0U2NRNW5IWKHU0BR1GPMA4JVKZSF5G' # your Foursquare ID
CLIENT_SECRET = 'YDGU0MYVNGQW151N3OPTHU2Y4BDB0JZWIWBD41DTEWTOORWF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius=500
LIMIT=100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OBQ0UO55QCKZIYTMPB0U2NRNW5IWKHU0BR1GPMA4JVKZSF5G
CLIENT_SECRET:YDGU0MYVNGQW151N3OPTHU2Y4BDB0JZWIWBD41DTEWTOORWF


In [15]:
import requests

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
toronto_venues = getNearbyVenues(names=Toronto_df['Neighbourhood'],
                                   latitudes=Toronto_df['Latitude'],
                                   longitudes=Toronto_df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [18]:
toronto_venues.head(5)

Neighbourhood  Neighbourhood Latitude  Neighbourhood Longitude  \
0         Parkwoods               43.753259               -79.329656   
1         Parkwoods               43.753259               -79.329656   
2  Victoria Village               43.725882               -79.315572   
3  Victoria Village               43.725882               -79.315572   
4  Victoria Village               43.725882               -79.315572   

                    Venue  Venue Latitude  Venue Longitude  \
0         Brookbanks Park       43.751976       -79.332140   
1           Variety Store       43.751974       -79.333114   
2  Victoria Village Arena       43.723481       -79.315635   
3               Portugril       43.725819       -79.312785   
4             Tim Hortons       43.725517       -79.313103   

          Venue Category  
0                   Park  
1      Food & Drink Shop  
2           Hockey Arena  
3  Portuguese Restaurant  
4            Coffee Shop

In [19]:
toronto_venues.shape

(2129, 7)

In [20]:
toronto_venues.groupby('Neighbourhood').count()

Neighbourhood Latitude  \
Neighbourhood                                                                                                                                                    
Agincourt                                                                                                                                                    5   
Alderwood, Long Branch                                                                                                                                       8   
Bathurst Manor, Wilson Heights, Downsview North                                                                                                             20   
Bayview Village                                                                                                                                              4   
Bedford Park, Lawrence Manor East                                                                                                                           24   
Berczy Park                                                                                                                                                 58   
Birch Cliff, Cliffside West                                                                                                                                  4   
Brockton, Parkdale Village, Exhibition Place                                                                                                                22   
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto                                                                        19   
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport                                                  15   
Caledonia-Fairbanks                                                                                                                                          4   
Canada Post Gateway Processing Centre                                                                                                                       13   
Cedarbrae                                                                                                                                                    9   
Central Bay Street                                                                                                                                          65   
Christie                                                                                                                                                    17   
Church and Wellesley                                                                                                                                        75   
Clarks Corners, Tam O'Shanter, Sullivan                                                                                                                     12   
Cliffside, Cliffcrest, Scarborough Village West                                                                                                              3   
Commerce Court, Victoria Hotel                                                                                                                             100   
Davisville                                                                                                                                                  35   
Davisville North                                                                                                                                             8   
Del Ray, Mount Dennis, Keelsdale and Silverthorn                                                                                                             5   
Don Mills                                                                                                                                                   25   
Dorset Park, Wexford Heights, Scarborough Town Centre                                 

In [21]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot.head()

Accessories Store  Afghan Restaurant  Airport  Airport Food Court  \
0                  0                  0        0                   0   
1                  0                  0        0                   0   
2                  0                  0        0                   0   
3                  0                  0        0                   0   
4                  0                  0        0                   0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  Aquarium  Art Gallery  Art Museum  Arts & Crafts Store  \
0             0         0            0           0                    0   
1             0         0            0           0                    0   
2             0         0            0           0                    0   
3             0         0            0           0                    0   
4             0         0            0           0                    0   

   Asian Restaurant  Athletics & Sports  Auto Garage  Auto Workshop  \
0                 0                   0            0              0   
1                 0                   0            0              0   
2                 0                   0            0              0   
3                 0                   0            0              0   
4                 0                   0            0              0   

   BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  Baseball Field  \
0          0           0           0       0     0    0               0   
1          0           0           0       0     0    0               0   
2          0           0           0       0     0    0               0   
3          0           0           0       0     0    0               0   
4          0           0           0       0     0    0               0   

   Baseball Stadium  Basketball Stadium  Beach  Bed & Breakfast  Beer Bar  \
0                 0                   0      0                0         0   
1                 0                   0      0                0         0   
2                 0                   0      0                0         0   
3                 0                   0      0                0         0   
4                 0                   0      0                0         0   

   Beer Store  Belgian Restaurant  Bike Shop  Bistro  Boat or Ferry  \
0           0                   0          0       0              0   
1           0                   0          0       0              0   
2           0                   0          0       0              0   
3           0                   0          0       0              0   
4           0                   0          0       0              0   

   Bookstore  Boutique  Brazilian Restaurant  Breakfast Spot  Brewery  \
0          0         0                     0               0        0   
1          0         0                     0               0        0   
2          0         0                     0               0        0   
3          0         0                     0               0        0   
4          0         0                     0               0        0   

   Bridal Shop  Bubble Tea Shop  Building  Burger Joint  Burrito Place  \
0            0                0         0             0              0   
1            0                0         0             0              0   
2            0                0         0             0              0   
3            0                0         0             0              0   
4            0                0         0             0              0   

   B

In [22]:
toronto_onehot.shape

(2129, 269)

In [23]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

Neighbourhood  \
0                                                                                                                                Agincourt   
1                                                                                                                   Alderwood, Long Branch   
2                                                                                          Bathurst Manor, Wilson Heights, Downsview North   
3                                                                                                                          Bayview Village   
4                                                                                                        Bedford Park, Lawrence Manor East   
5                                                                                                                              Berczy Park   
6                                                                                                              Birch Cliff, Cliffside West   
7                                                                                             Brockton, Parkdale Village, Exhibition Place   
8                                                     Business reply mail Processing Centre, South Central Letter Processing Plant Toronto   
9                               CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport   
10                                                                                                                     Caledonia-Fairbanks   
11                                                                                                   Canada Post Gateway Processing Centre   
12                                                                                                                               Cedarbrae   
13                                                                                                                      Central Bay Street   
14                                                                                                                                Christie   
15                                                                                                                    Church and Wellesley   
16                                                                                                 Clarks Corners, Tam O'Shanter, Sullivan   
17                                                                                         Cliffside, Cliffcrest, Scarborough Village West   
18                                                                                                          Commerce Court, Victoria Hotel   
19                                                                                                                              Davisville   
20                                                                                                                        Davisville North   
21                                                                                        Del Ray, Mount Dennis, Keelsdale and Silverthorn   
22                                                                                                                               Don Mills   
23                                                                                   Dorset Park, Wexford Heights, Scarborough Town Centre   
24                                                                                                                               Downsview   
25                                                                                                            Dufferin, Dovercourt Village   
26                                                                                           East Toronto, Broadview North (Old East York)   
27                                                                           Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood   
28     

In [24]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot   0.2
1                     Lounge   0.2
2             Clothing Store   0.2
3               Skating Rink   0.2
4  Latin American Restaurant   0.2


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.25
1           Pub  0.12
2           Gym  0.12
3  Dance Studio  0.12
4   Coffee Shop  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
                venue  freq
0         Coffee Shop  0.10
1                Bank  0.10
2    Sushi Restaurant  0.05
3       Shopping Mall  0.05
4  Frozen Yogurt Shop  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1   Chinese Restaurant  0.25
2                 Bank  0.25
3                 Café  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.08
1         Coffee Shop  0.08
2          Restaurant  0.08
3      Sandwich Place  0.

4  Italian Restaurant  0.03


----Regent Park, Harbourfront----
         venue  freq
0  Coffee Shop  0.16
1       Bakery  0.07
2         Park  0.07
3          Pub  0.07
4      Theater  0.05


----Richmond, Adelaide, King----
         venue  freq
0  Coffee Shop  0.11
1   Restaurant  0.05
2         Café  0.05
3          Gym  0.04
4        Hotel  0.03


----Rosedale----
                venue  freq
0                Park  0.50
1          Playground  0.25
2               Trail  0.25
3   Accessories Store  0.00
4  Mexican Restaurant  0.00


----Roselawn----
                       venue  freq
0                     Garden  0.33
1                       Pool  0.33
2               Home Service  0.33
3  Middle Eastern Restaurant  0.00
4        Moroccan Restaurant  0.00


----Rouge Hill, Port Union, Highland Creek----
                             venue  freq
0                              Bar   1.0
1                Accessories Store   0.0
2        Middle Eastern Restaurant   0.0
3              Monum

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

Neighbourhood 1st Most Common Venue  \
0                                        Agincourt        Clothing Store   
1                           Alderwood, Long Branch           Pizza Place   
2  Bathurst Manor, Wilson Heights, Downsview North           Coffee Shop   
3                                  Bayview Village                  Café   
4                Bedford Park, Lawrence Manor East           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue      4th Most Common Venue  \
0                Lounge        Breakfast Spot  Latin American Restaurant   
1                   Pub                   Gym               Dance Studio   
2                  Bank     Mobile Phone Shop         Frozen Yogurt Shop   
3   Japanese Restaurant                  Bank         Chinese Restaurant   
4            Restaurant        Sandwich Place         Italian Restaurant   

  5th Most Common Venue        6th Most Common Venue 7th Most Common Venue  \
0          Skating Rink  Eastern European Restaurant   Distribution Center   
1        Sandwich Place                  Coffee Shop                  Pool   
2         Shopping Mall               Sandwich Place                 Diner   
3        Discount Store          Distribution Center               Dog Run   
4      Greek Restaurant              Thai Restaurant              Pharmacy   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0               Dog Run      Doner Restaurant             Donut Shop  
1           Yoga Studio                 Diner         Discount Store  
2            Restaurant           Supermarket       Sushi Restaurant  
3      Doner Restaurant            Donut Shop            Yoga Studio  
4           Pizza Place                   Pub                   Café

In [27]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4,
       1, 4, 1, 4, 4, 2, 0, 4, 1, 4, 4, 4, 4, 4, 2, 4, 4, 1, 4, 4, 4, 1,
       4, 3, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 1])

In [28]:
neighbourhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

toronto_merged = Toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

Postal Code           Borough                                Neighbourhood  \
0         M3A        North York                                    Parkwoods   
1         M4A        North York                             Victoria Village   
2         M5A  Downtown Toronto                    Regent Park, Harbourfront   
3         M6A        North York             Lawrence Manor, Lawrence Heights   
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  Cluster_Labels 1st Most Common Venue  \
0  43.753259 -79.329656             1.0     Food & Drink Shop   
1  43.725882 -79.315572             4.0           Pizza Place   
2  43.654260 -79.360636             4.0           Coffee Shop   
3  43.718518 -79.464763             4.0        Clothing Store   
4  43.662301 -79.389494             4.0           Coffee Shop   

  2nd Most Common Venue        3rd Most Common Venue 4th Most Common Venue  \
0                  Park  Eastern European Restaurant        Discount Store   
1     French Restaurant                  Coffee Shop          Hockey Arena   
2                Bakery                          Pub                  Park   
3     Accessories Store       Furniture / Home Store           Coffee Shop   
4                 Diner             Sushi Restaurant   Fried Chicken Joint   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0    Distribution Center               Dog Run      Doner Restaurant   
1  Portuguese Restaurant          Intersection           Yoga Studio   
2         Breakfast Spot                  Café               Theater   
3            Event Space    Miscellaneous Shop              Boutique   
4          Smoothie Shop         Burrito Place        Sandwich Place   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0             Donut Shop             Drugstore            Yoga Studio  
1       Doner Restaurant                 Diner         Discount Store  
2            Yoga Studio            Beer Store             Shoe Store  
3  Vietnamese Restaurant       Coworking Space         Farmers Market  
4                   Café              Beer Bar     Italian Restaurant

In [29]:
toronto_merged=toronto_merged.dropna()

In [30]:
toronto_merged['Cluster_Labels'] = toronto_merged.Cluster_Labels.astype(int)

In [31]:
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### cluster 3

In [32]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster_Labels 1st Most Common Venue 2nd Most Common Venue  \
32  Scarborough               0            Playground           Yoga Studio   
85  Scarborough               0                  Park            Playground   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
32             Drugstore                 Diner        Discount Store   
85      Sculpture Garden            Donut Shop                 Diner   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
32   Distribution Center               Dog Run      Doner Restaurant   
85        Discount Store   Distribution Center               Dog Run   

   9th Most Common Venue       10th Most Common Venue  
32            Donut Shop  Eastern European Restaurant  
85      Doner Restaurant                    Drugstore

###  cluster 2

In [33]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster_Labels 1st Most Common Venue  \
0         North York               1     Food & Drink Shop   
21              York               1                  Park   
35         East York               1                  Park   
61   Central Toronto               1                  Park   
66        North York               1                  Park   
77         Etobicoke               1                  Park   
83   Central Toronto               1                  Park   
91  Downtown Toronto               1                  Park   
98         Etobicoke               1                  Park   

   2nd Most Common Venue        3rd Most Common Venue 4th Most Common Venue  \
0                   Park  Eastern European Restaurant        Discount Store   
21         Women's Store                         Pool             Drugstore   
35     Convenience Store                Metro Station           Yoga Studio   
61              Bus Line                  Swim School            Donut Shop   
66     Convenience Store                  Yoga Studio             Drugstore   
77        Sandwich Place                  Yoga Studio             Drugstore   
83           Yoga Studio                    Drugstore                 Diner   
91            Playground                        Trail      Doner Restaurant   
98                 River                   Smoke Shop            Donut Shop   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0    Distribution Center               Dog Run      Doner Restaurant   
21                 Diner        Discount Store   Distribution Center   
35             Drugstore   Distribution Center               Dog Run   
61        Discount Store   Distribution Center               Dog Run   
66        Discount Store   Distribution Center               Dog Run   
77        Discount Store   Distribution Center               Dog Run   
83        Discount Store   Distribution Center               Dog Run   
91    Dim Sum Restaurant                 Diner        Discount Store   
98                 Diner        Discount Store   Distribution Center   

   8th Most Common Venue 9th Most Common Venue       10th Most Common Venue  
0             Donut Shop             Drugstore                  Yoga Studio  
21               Dog Run      Doner Restaurant                   Donut Shop  
35      Doner Restaurant            Donut Shop            Electronics Store  
61      Doner Restaurant             Drugstore                        Field  
66      Doner Restaurant            Donut Shop            Electronics Store  
77      Doner Restaurant            Donut Shop  Eastern European Restaurant  
83      Doner Restaurant            Donut Shop  Eastern European Restaurant  
91   Distribution Center               Dog Run                   Donut Shop  
98               Dog Run      Doner Restaurant                    Drugstore

### cluster 3

In [34]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster_Labels 1st Most Common Venue 2nd Most Common Venue  \
6    Scarborough               2  Fast Food Restaurant           Yoga Studio   
57    North York               2        Baseball Field           Yoga Studio   
101    Etobicoke               2        Baseball Field          Home Service   

           3rd Most Common Venue        4th Most Common Venue  \
6             Dim Sum Restaurant               Farmers Market   
57   Eastern European Restaurant          Distribution Center   
101                  Yoga Studio  Eastern European Restaurant   

    5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
6      Falafel Restaurant           Event Space  Ethiopian Restaurant   
57                Dog Run      Doner Restaurant            Donut Shop   
101   Distribution Center               Dog Run      Doner Restaurant   

    8th Most Common Venue        9th Most Common Venue 10th Most Common Venue  
6       Electronics Store  Eastern European Restaurant              Drugstore  
57              Drugstore            Electronics Store      Fish & Chips Shop  
101            Donut Shop                    Drugstore      Electronics Store

### cluster 4

In [35]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster_Labels 1st Most Common Venue 2nd Most Common Venue  \
12  Scarborough               3                   Bar           Yoga Studio   

          3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
12  Eastern European Restaurant   Distribution Center               Dog Run   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
12      Doner Restaurant            Donut Shop             Drugstore   

   9th Most Common Venue 10th Most Common Venue  
12     Electronics Store      Fish & Chips Shop

### cluster 5

In [36]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster_Labels   1st Most Common Venue  \
1          North York               4             Pizza Place   
2    Downtown Toronto               4             Coffee Shop   
3          North York               4          Clothing Store   
4    Downtown Toronto               4             Coffee Shop   
7          North York               4                     Gym   
8           East York               4             Pizza Place   
9    Downtown Toronto               4          Clothing Store   
10         North York               4                    Park   
13         North York               4                     Gym   
14          East York               4                    Park   
15   Downtown Toronto               4             Coffee Shop   
16               York               4            Tennis Court   
17          Etobicoke               4               Pet Store   
18        Scarborough               4            Intersection   
19       East Toronto               4            Neighborhood   
20   Downtown Toronto               4             Coffee Shop   
22        Scarborough               4             Coffee Shop   
23          East York               4             Coffee Shop   
24   Downtown Toronto               4             Coffee Shop   
25   Downtown Toronto               4           Grocery Store   
26        Scarborough               4        Hakka Restaurant   
27         North York               4                 Dog Run   
28         North York               4             Coffee Shop   
29          East York               4       Indian Restaurant   
30   Downtown Toronto               4             Coffee Shop   
31       West Toronto               4                  Bakery   
33         North York               4          Clothing Store   
34         North York               4  Furniture / Home Store   
36   Downtown Toronto               4             Coffee Shop   
37       West Toronto               4                     Bar   
38        Scarborough               4          Discount Store   
39         North York               4                    Café   
40         North York               4           Grocery Store   
41       East Toronto               4        Greek Restaurant   
42   Downtown Toronto               4             Coffee Shop   
43       West Toronto               4                    Café   
44        Scarborough               4                  Bakery   
46         North York               4           Grocery Store   
47       East Toronto               4                    Park   
48   Downtown Toronto               4             Coffee Shop   
49         North York               4                    Park   
50         North York               4             Pizza Place   
51        Scarborough               4                   Motel   
53         North York               4           Grocery Store   
54       East Toronto               4                    Café   
55         North York               4             Coffee Shop   
56               York               4          Discount Store   
58        Scarborough               4         College Stadium   
59         North York               4        Ramen Restaurant   
60         North York               4           Grocery Store   
62    Central Toronto               4                  Garden   
63               York               4             Pizza Place   
65        Scarborough               4       Indian Restaurant   
67    Central Toronto               4    Gym / Fitness Center   
68    Central Toronto               4           Jewelry Store   
69       West Toronto               4         Thai Restaurant   
70          Etobicoke               4             Pizza Place   
71        Scarborough               4          Breakfast Spot   
72         North York               4                Pharmacy   
73    Central Toronto               4          Clothing Store   
74    Central Toronto               4          Sa